In [15]:
import random

In [33]:
arrival_probabilities = [
    0.0094,
    0.0094,
    0.0094,
    0.0094,
    0.0094,
    0.0094,
    0.0094,
    0.0094,
    0.0283,
    0.0283,
    0.0566,
    0.0566,
    0.0566,
    0.0755,
    0.0755,
    0.0755,
    0.1038,
    0.1038,
    0.1038,
    0.0472,
    0.0472,
    0.0472,
    0.0094,
    0.0094
]

charging_demand_probabilities = [
    (0.3431, 0),
    (0.0490, 5),
    (0.0980, 10),
    (0.1176, 20),
    (0.0882, 30),
    (0.1176, 50),
    (0.1078, 100),
    (0.0490, 200),
    (0.0294, 300)
]

In [34]:
def simulate_chargepoint(num_ticks, chargepoint_states, total_energy_per_tick, power_demand_per_tick, charging_power, minutes_per_tick, kwh_per_100km):
    total_energy = 0

    for tick in range(num_ticks):
        if chargepoint_states[tick] is False:
            hour_of_day = (tick // (60 // minutes_per_tick)) % 24
            if random.random() < arrival_probabilities[hour_of_day]:
                charging_demand = random.choices(charging_demand_probabilities, weights=[p[0] for p in charging_demand_probabilities])[0][1]
                if charging_demand > 0:
                    charge_ticks_remaining = int(charging_demand / 100 * kwh_per_100km / charging_power * 60 / minutes_per_tick)
                    for i in range(tick, min(tick + charge_ticks_remaining, num_ticks)):
                        chargepoint_states[i] = True
                        total_energy_per_tick[i] += charging_power * minutes_per_tick / 60
                        power_demand_per_tick[i] += charging_power
        else:
            chargepoint_states[tick] = False
            total_energy += charging_power * minutes_per_tick / 60

    return total_energy

In [35]:
def run(num_chargepoints, charging_power, minutes_per_tick, kwh_per_100km):
    ticks_per_hour = 60 // minutes_per_tick
    ticks_per_day = 24 * ticks_per_hour
    ticks_per_year = 365 * ticks_per_day

    chargepoint_states = {tick: False for tick in range(ticks_per_year)}
    total_energy_per_tick = {tick: 0 for tick in range(ticks_per_year)}
    power_demand_per_tick = {tick: 0 for tick in range(ticks_per_year)}

    for _ in range(num_chargepoints):
        simulate_chargepoint(ticks_per_year, chargepoint_states, total_energy_per_tick, power_demand_per_tick, charging_power, minutes_per_tick, kwh_per_100km)

    total_energy_consumed = sum(total_energy_per_tick.values())
    max_power_demand = max(power_demand_per_tick.values())
    theoretical_max_power_demand = num_chargepoints * charging_power
    concurrency_factor = max_power_demand / theoretical_max_power_demand

    print(f"Total energy consumed: {total_energy_consumed:.2f} kWh")
    print(f"Theoretical maximum power demand: {theoretical_max_power_demand:.2f} kW")
    print(f"Actual maximum power demand: {max_power_demand:.2f} kW")
    print(f"Concurrency factor: {concurrency_factor:.2f}")

In [36]:
NUM_CHARGEPOINTS = 20
CHARGING_POWER = 11  # kW
MINUTES_PER_TICK = 15
KWH_PER_100KM = 18

run(NUM_CHARGEPOINTS, CHARGING_POWER, MINUTES_PER_TICK, KWH_PER_100KM)

Total energy consumed: 168806.00 kWh
Theoretical maximum power demand: 220.00 kW
Actual maximum power demand: 110.00 kW
Concurrency factor: 0.50
